In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform pymupdf

  Using cached google_cloud_aiplatform-1.43.0-py2.py3-none-any.whl.metadata (27 kB)
Using cached google_cloud_aiplatform-1.43.0-py2.py3-none-any.whl (4.1 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.25.0
    Uninstalling google-cloud-aiplatform-1.25.0:
      Successfully uninstalled google-cloud-aiplatform-1.25.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [3]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
LOCATION = "us-east4"

In [1]:
from IPython.display import Markdown, display
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part,
)
from vertexai.language_models import TextEmbeddingModel

In [2]:
model = GenerativeModel("gemini-pro-vision")
text_embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [3]:
contents = """
Summarize the following text in no more than 5 sentences, in English, only using the text. 

Text: {text}
SUMMARY: 
"""
print(contents.format(text='123'))


Summarize the following text in no more than 5 sentences, in English, only using the text. 

Text: 123
SUMMARY: 



In [4]:
Contents = str | list[str | Image | Part]

def generate_content(
    model: GenerativeModel,
    contents: Contents,
    temperature: float = 0.0,
    top_p: float = 0.0,
    top_k: int = 1,
) -> list[GenerationResponse]:
    """Call the Vertex AI Gemini API.

    Default parameters have low randomness in this notebook for consistency across calls.
    """
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        candidate_count=1,
        max_output_tokens=2048,
    )

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        stream=True,
    )

    # In streaming mode, multiple GenerationResponse can be generated
    # In unary (non-streaming) mode, a single GenerationResponse is returned
    return [responses] if isinstance(responses, GenerationResponse) else list(responses)


In [5]:
def print_responses(responses: list[GenerationResponse], as_markdown: bool = True):
    """Print the full responses."""
    # Consolidate the text
    text = "".join(
        part.text
        for response in responses
        for part in response.candidates[0].content.parts
    )
    # Remove potential leading/trailing spaces
    text = text.strip()

    print(" Start of responses ".center(80, "-"))
    print(text)
    print(" End of responses ".center(80, "-"))
    print("")

In [6]:
from pathlib import Path
def get_file_id(path: Path) -> str:
    return path.stem

def generate_res(data: str) -> str:
    responses = generate_content(model, contents.format(text=data))
    responses_post = "".join(
        part.text
        for response in responses
        for part in response.candidates[0].content.parts
    )
    return responses_post

In [7]:
from google.cloud import storage


def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    bucket_name = bucket_name

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    blob_names = []
    for blob in blobs:
        if blob.name.endswith('.txt'):
            print(blob.name)
            break
            blob_names.append(blob.name)
    
    return blob_names

blob_names = list_blobs("lloyds-genai24lon-2701-bucket")
blob_names[0]
    

'text_extracts/DRN-1000090_2019-01-03.txt'

In [8]:
def write_read(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Mode can be specified as wb/rb for bytes mode.
    # See: https://docs.python.org/3/library/io.html
    with blob.open("r") as f:
        data = f.read()
    return data

In [12]:
all_res = {}
for i, f in enumerate(Path("./text").glob('*.txt')):
    if i == 50:
        break
    data = f.read_text()
    res = generate_res(data)
    file_id = get_file_id(f)
    all_res[file_id] = res


In [ ]:
all_res = {}
for i, f in enumerate(blob_names):
    if 

In [13]:
all_res

{'DRN-1074996_2020-04-15': "Mr. A's car was stolen, and the thief was convicted. However, UKI recorded the claim as a fault claim because they could not recover all costs from the third party due to lack of information from the police. The investigator and Ombudsman agree that UKI acted fairly in recording the claim as a fault claim, as it does not imply liability but reflects the inability to recover costs. Despite Mr. A's frustration, the Ombudsman cannot require UKI to amend the record to a non-fault claim. The complaint is not upheld.",
 'DRN-2081709_2020-08-18': "Mrs. T complained that AXA France IARD mis-sold her payment protection insurance (PPI) in 1996. AXA denied liability, claiming Mrs. T had a fair choice to purchase the PPI and was provided with sufficient information to make an informed decision. The ombudsman considered the evidence and determined that Mrs. T likely agreed to the PPI, despite not fully understanding it. The ombudsman also found that the PPI could have be

In [14]:
import pandas as pd
df = pd.DataFrame(all_res.items(), columns=['file_name', 'text_summary'])

In [15]:
df[['id', 'date']] = df.file_name.str.split('_').tolist()
df

,file_name,text_summary,id,date
0,DRN-1074996_2020-04-15,"Mr. A's car was stolen, and the thief was conv...",DRN-1074996,2020-04-15
1,DRN-2081709_2020-08-18,Mrs. T complained that AXA France IARD mis-sol...,DRN-2081709,2020-08-18
2,DRN-1645227_2020-08-12,- Company S complained about errors made by HP...,DRN-1645227,2020-08-12
3,DRN-2202979_2020-11-13,- Mrs. R alleges that an interest-only mortgag...,DRN-2202979,2020-11-13
4,DRN-1624150_2020-05-04,Mr. W filed a chargeback claim with Barclays a...,DRN-1624150,2020-05-04
5,DRN-3176047_2021-12-15,Miss D complained to the Financial Ombudsman S...,DRN-3176047,2021-12-15
6,DRN-1081199_2020-08-26,Mrs. V's insurance claim was initially delayed...,DRN-1081199,2020-08-26
7,DRN-3222796_2022-02-22,Mrs. M complained about Zurich Insurance's han...,DRN-3222796,2022-02-22
8,DRN-1917278_2020-06-23,Mr. H complained that Elevate Credit (Sunny) l...,DRN-1917278,2020-06-23
9,DRN-3392770_2022-05-03,Mr. and Mrs. A filed a complaint against Royal...,DRN-3392770,2022-05-03


In [35]:
df['text_summary'] = df['text_summary'].map(lambda x: x.replace('-',''))
df['text_summary'] = df['text_summary'].map(lambda x: x.replace('\n',''))

In [36]:
df.text_summary[13]

" Miss C purchased a used car through a hire purchase agreement from Oodle Financial Services in September 2019, but experienced various issues with the car's quality. An independent inspection revealed frontal damage to the car, which Miss C believes was present at the point of sale. Oodle initially rejected Miss C's complaint, but the Financial Ombudsman Service upheld it, finding that the car was not of satisfactory quality. The Ombudsman ordered Oodle to end the hire purchase agreement, refund Miss C's payments, and pay her compensation for distress and inconvenience. Oodle must also remove any adverse information about the agreement from Miss C's credit file."

In [37]:
df.to_csv('fos_summary.csv')

In [17]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")

In [18]:
client.create_dataset("fos_dataset")

Dataset(DatasetReference('qwiklabs-gcp-02-44cffb82aab4', 'fos_dataset'))

In [19]:
# create a new table in that dataset ()
client.create_table(f"{PROJECT_ID}.fos_dataset.fos_table")

Table(TableReference(DatasetReference('qwiklabs-gcp-02-44cffb82aab4', 'fos_dataset'), 'fos_table'))

In [38]:
# some variables
filename = 'fos_summary.csv' # this is the file path to your csv
dataset_id = 'fos_dataset'
table_id = 'fos_table'

# tell the client everything it needs to know to upload our csv
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.autodetect = True

# load the csv into bigquery
with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

# looks like everything worked :)
print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

Loaded 50 rows into fos_dataset:fos_table.


In [40]:
# query (you won't want to use SELECT * unless your dataset is very small)
query = f""" SELECT * 
        FROM `{PROJECT_ID}.fos_dataset.fos_table`"""

# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
data = query_job.to_dataframe()

In [41]:
data

,int64_field_0,file_name,text_summary,id,date,result
0,0,DRN-1074996_2020-04-15,"Mr. A's car was stolen, and the thief was conv...",DRN-1074996,2020-04-15,"Mr. A's car was stolen, and the thief was conv..."
1,1,DRN-2081709_2020-08-18,Mrs. T complained that AXA France IARD missold...,DRN-2081709,2020-08-18,Mrs. T complained that AXA France IARD missold...
2,2,DRN-1645227_2020-08-12,Company S complained about errors made by HPS...,DRN-1645227,2020-08-12,Company S complained about errors made by HPS...
3,3,DRN-2202979_2020-11-13,Mrs. R alleges that an interestonly mortgage ...,DRN-2202979,2020-11-13,Mrs. R alleges that an interestonly mortgage ...
4,4,DRN-1624150_2020-05-04,Mr. W filed a chargeback claim with Barclays a...,DRN-1624150,2020-05-04,Mr. W filed a chargeback claim with Barclays a...
5,5,DRN-3176047_2021-12-15,Miss D complained to the Financial Ombudsman S...,DRN-3176047,2021-12-15,Miss D complained to the Financial Ombudsman S...
6,6,DRN-1081199_2020-08-26,Mrs. V's insurance claim was initially delayed...,DRN-1081199,2020-08-26,Mrs. V's insurance claim was initially delayed...
7,7,DRN-3222796_2022-02-22,Mrs. M complained about Zurich Insurance's han...,DRN-3222796,2022-02-22,Mrs. M complained about Zurich Insurance's han...
8,8,DRN-1917278_2020-06-23,Mr. H complained that Elevate Credit (Sunny) l...,DRN-1917278,2020-06-23,Mr. H complained that Elevate Credit (Sunny) l...
9,9,DRN-3392770_2022-05-03,Mr. and Mrs. A filed a complaint against Royal...,DRN-3392770,2022-05-03,Mr. and Mrs. A filed a complaint against Royal...


In [55]:
query_prompt = prompt = [
    """
    You are an expert in converting English questions to SQL query!
    The SQL database has the name fos_dataset.fos_table and has the following columns - file_name, text_summary, 
    id, date 
    
    For example,
    
    Example 1 - How many entries of records are present?, 
    the SQL command will be something like this SELECT COUNT(*) FROM fos_dataset.fos_table;
    
    Example 2 - Tell me what does DRN-2823174 complain about?, 
    the SQL command will be something like this SELECT * FROM fos_dataset.fos_table
    where id="DRN-2823174"; 
    
    also the sql code should not have ``` in beginning or end and sql word in output
    
    Please response this in a polite way

    """
]


In [43]:
question = "How many entries of records are present?"

In [46]:
def get_gemini_response(question, prompt):
    response=model.generate_content([prompt[0],question])
    return response.text

In [56]:
get_gemini_response(question, prompt)

'SELECT COUNT(*) FROM fos_dataset.fos_table;'

In [54]:
# query (you won't want to use SELECT * unless your dataset is very small)
query = f""" SELECT COUNT(*) 
        FROM `{PROJECT_ID}.fos_dataset.fos_table`"""

# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
data = query_job.to_dataframe()
data

,f0_
0,50
